"""
Yelp Latin American Restaurants Dataset
---------------------------------------
This script queries the Yelp Fusion API for Latin American or Salvadoran restaurants
across several U.S. cities, compiles results, and saves them as a CSV.

Author: Daniela Binns
"""

In [2]:
# Import nessesary packages

import requests
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
API_KEY = ""
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [4]:
# Choose cities
cities = [
    "Atlanta, GA",
    "Miami, FL",
    "Los Angeles, CA",
    "New York, NY",
    "Houston, TX",
    "Salt Lake City, UT",
    "San Francisco, CA",
    "Chicago, IL",
    "Washington, DC",
    "Dallas, TX",
    "Provo, UT"
]

In [5]:
# Define search
SEARCH_TERM = "Latin American"
LIMIT = 50  # Yelp API allows up to 50 results per request
MAX_RESULTS_PER_CITY = 200  # Yelp caps total at 1000

In [6]:
def fetch_yelp_data(location, term=SEARCH_TERM, limit=LIMIT, max_results=MAX_RESULTS_PER_CITY):
    """Fetch restaurant data from Yelp API for a given city."""
    all_results = []
    for offset in range(0, max_results, limit):
        url = "https://api.yelp.com/v3/businesses/search"
        params = {
            "term": term,
            "location": location,
            "categories": "restaurants",
            "limit": limit,
            "offset": offset,
            "sort_by": "rating"
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print(f"Error {response.status_code} for {location} offset {offset}")
            break
        data = response.json()
        businesses = data.get("businesses", [])
        if not businesses:
            break
        all_results.extend(businesses)
        time.sleep(0.5)
    return all_results

In [7]:
def parse_business(b, city):
    """Flatten Yelp business JSON into a flat dictionary."""
    return {
        "id": b.get("id"),
        "name": b.get("name"),
        "rating": b.get("rating"),
        "review_count": b.get("review_count"),
        "price": b.get("price", None),
        "city": city,
        "address": " ".join(b.get("location", {}).get("display_address", [])),
        "latitude": b.get("coordinates", {}).get("latitude"),
        "longitude": b.get("coordinates", {}).get("longitude"),
        "category": ", ".join([c["title"] for c in b.get("categories", [])]),
        "phone": b.get("display_phone"),
        "url": b.get("url")
    }

In [8]:
def main():
    all_records = []

    for city in tqdm(cities, desc="Fetching cities"):
        results = fetch_yelp_data(city)
        parsed = [parse_business(b, city) for b in results]
        all_records.extend(parsed)
        print(f"{len(parsed)} results from {city}")

    df = pd.DataFrame(all_records)

    # Basic cleaning
    df.drop_duplicates(subset="id", inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Convert price to numeric scale ($=1, $$=2, etc.)
    df["price_level"] = df["price"].map({"$": 1, "$$": 2, "$$$": 3, "$$$$": 4})

    # Save dataset
    df.to_csv("latin_american_restaurants.csv", index=False)
    print(f"\n💾 Saved {len(df)} restaurants to latin_american_restaurants.csv")

    # Preview
    print(df.head())


In [9]:
if __name__ == "__main__":
    main()

Fetching cities:   9%|▉         | 1/11 [00:07<01:18,  7.86s/it]

200 results from Atlanta, GA


Fetching cities:  18%|█▊        | 2/11 [00:14<01:04,  7.22s/it]

200 results from Miami, FL


Fetching cities:  27%|██▋       | 3/11 [00:21<00:56,  7.07s/it]

...

200 results from Washington, DC


Fetching cities:  91%|█████████ | 10/11 [01:06<00:06,  6.71s/it]

200 results from Dallas, TX


Fetching cities: 100%|██████████| 11/11 [01:08<00:00,  6.22s/it]

19 results from Provo, UT

💾 Saved 1895 restaurants to latin_american_restaurants.csv
                       id                       name  rating  review_count  \
0  GQwdg3e2MzPORQFf7r7Kug  Mrs. Rosa's Latin Kitchen     5.0             6   
1  LMuXS8dhMHqu7qdseUc35A         Buena Papa Fry Bar     4.9            16   
2  imK2KdIySXdbj0FI8YPD8A                La Tropical     4.8            13   
3  n20rQSzceONC3eQRnprmhQ                Arepa Grill     4.7            45   
4  tuNjPUE2QF1FroLFYwLAWg                 La Semilla     4.7           199   

  price         city                                            address  \
0  None  Atlanta, GA  3940 Peachtree Industrial Blvd Ste C Duluth, G...   
1  None  Atlanta, GA             1245 Glenwood Ave SE Atlanta, GA 30316   
2  None  Atlanta, GA        1235 Chattahoochee Ave NW Atlanta, GA 30318   
3  None  Atlanta, GA               4166 Buford Hwy NE Atlanta, GA 30345   
4    $$  Atlanta, GA        780 Memorial Dr SE Ste 4A Atlanta, GA 3031